# Twitter analysis 

<center><img src="https://techcrunch.com/wp-content/uploads/2017/12/gettyimages-876768474.jpg?w=730&crop=1"></center>

Analyze the sentiment of tweets on some query search, your model should learn from [this dataset](https://www.kaggle.com/kazanova/sentiment140)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
query = "egypt"

In [4]:
!pip install pydrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
downloaded = drive.CreateFile({'id':"1CZtBvbjvUvEbVuFyMVGzmIkU1uFWv_RQ"})   # replace the id with id of file you want to access
downloaded.GetContentFile('Copy of training_twitter.csv') 

# Start by building your sentiment analysis classifier

In [0]:
# imports
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
%matplotlib inline

In [44]:
# reading the data
cols = ["Sentiment","ID","Date","Query","Username","Tweet"]
dataset = pd.read_csv("Copy of training_twitter.csv", encoding='latin-1',delimiter='","',engine='python')
dataset.head()

,"""0",1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"""
0,"""0",1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,"""0",1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,"""0",1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,"my whole body feels itchy and like its on fire """
3,"""0",1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,"""0",1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,"@Kwesidei not the whole crew """


In [45]:
from sklearn.utils import shuffle
dataset = shuffle(dataset)
dataset

,"""0",1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"""
619288,"""0",2227865899,Thu Jun 18 13:43:49 PDT 2009,NO_QUERY,RenovatioNow,"Going this way is futile, i should have known ..."
753838,"""0",2287022737,Mon Jun 22 17:14:46 PDT 2009,NO_QUERY,starringSHORA,"RIP laptop no more computer for meee"""
48840,"""0",1677977432,Sat May 02 04:57:21 PDT 2009,NO_QUERY,misscee,"@feliciatemple awwwww leesh... """
152440,"""0",1932738216,Tue May 26 22:06:24 PDT 2009,NO_QUERY,AnnaAtomic,"im sick... my head hurts really bad..."""
3976,"""0",1468718005,Tue Apr 07 03:20:50 PDT 2009,NO_QUERY,BabaMzungu,Postponing my trip to Kenya. G/F going to TZ f...
...,...,...,...,...,...,...
736082,"""0",2264932335,Sun Jun 21 05:20:47 PDT 2009,NO_QUERY,xAbbysarahx,"Time to revise """
328460,"""0",2010609935,Tue Jun 02 17:40:16 PDT 2009,NO_QUERY,emacdanico,"@kaseyleboeuf i feel so hated """
878414,"""4",1685239043,Sat May 02 23:34:32 PDT 2009,NO_QUERY,clintlundie,"should have said &quot;tweeple&quot; """
1072135,"""4",1966502240,Fri May 29 17:53:08 PDT 2009,NO_QUERY,JonGuiro,I finally joined the twitter network! Yay! Hi ...


In [46]:
dataset = dataset.iloc[:500000,:]
dataset.columns = ["Sentiment","ID","Date","Query","Username","Tweet"]
dataset['Sentiment'] = dataset['Sentiment'].apply(lambda x: int(x[1:]))
dataset.info()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


<class 'pandas.core.frame.DataFrame'>
Int64Index: 500000 entries, 619288 to 259418
Data columns (total 6 columns):
Sentiment    500000 non-null int64
ID           500000 non-null int64
Date         500000 non-null object
Query        500000 non-null object
Username     500000 non-null object
Tweet        500000 non-null object
dtypes: int64(2), object(4)
memory usage: 26.7+ MB


In [0]:
def check_for_word(text):
    if query in text:
        return text
    else:
        return None

In [48]:
features = dataset.drop(columns=["ID","Date","Query","Username"])
features["Tweet"] = features["Tweet"]
goal = features["Sentiment"]
features = features.drop(columns=["Sentiment"])
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500000 entries, 619288 to 259418
Data columns (total 1 columns):
Tweet    500000 non-null object
dtypes: object(1)
memory usage: 7.6+ MB


In [0]:
import re
def normalize_text(text):
    
    # reduce all to lower case
    text = str(text).strip().lower()
    
    # remove non-alpha characters
    text = re.sub(r"[^a-z]", " ", text)
    
    return text.strip()

In [50]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
def stemming_text(text):
    words = text.split()
    words = [stemmer.stem(word.strip()) for word in words]
    return " ".join(words)

In [0]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
def lemm_text(text):
    words = text.split()
    words = [lemmatizer.lemmatize(word, pos='v') for word in words]
    return " ".join(words)

In [53]:
features["Tweet"] = features["Tweet"].apply(normalize_text)
features["Tweet"] = features["Tweet"].apply(stemming_text)
features["Tweet"] = features["Tweet"].apply(lemm_text)

features

,Tweet
619288,go thi way be futil i should have know better ...
753838,rip laptop no more comput for meee
48840,feliciatempl awwwww leesh
152440,im sick my head hurt realli bad
3976,postpon my trip to kenya g f go to tz for trai...
...,...
1398791,my new shoe make my leave foot stick out more ...
320542,some quick shutey on the train but now my mood...
1486053,out mahh sistaa cookout new jersey awww gonna ...
1401664,watch into the wood


In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', max_features=2000, min_df=20)

vectorizer.fit(features["Tweet"])

features = vectorizer.transform(features["Tweet"])

features = pd.DataFrame(
    features.astype(np.int32).toarray(), columns=vectorizer.get_feature_names())
features.head()

,abl,absolut,abt,ac,accept,access,accid,account,ace,ach,act,action,activ,actual,ad,adam,add,addict,address,admit,ador,adventur,advic,afford,afraid,afternoon,age,ago,agre,ah,aha,ahaha,ahead,ahh,ahhh,ahhhh,aim,ain,aint,air,...,wouldn,wouldnt,wow,wrap,write,wrong,wtf,www,xbox,xd,xo,xoxo,xx,xxx,xxxx,ya,yah,yall,yard,yay,ye,yea,yeah,year,yeh,yell,yep,yesterday,yey,yfrog,yo,york,young,youtub,youu,yr,yu,yum,yummi,yup
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features, goal,test_size=0.1, random_state = 0)

In [56]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

# clf1 = LogisticRegression(random_state=1)
clf2 = LinearSVC(random_state=1)
clf3 = GaussianNB()
clf4 = MultinomialNB()
# clf5 = DecisionTreeClassifier(max_depth=4)
# clf6 = KNeighborsClassifier(n_neighbors=10)
# clf7 = SVC(kernel='rbf', probability=True)

# eclf = VotingClassifier(estimators=[('lr', clf1), ('svc', clf2), ('gnb', clf3) , ('mnb', clf4) , ('dt', clf5) , ('knn', clf6) , ('svc', clf7)], voting='hard')
eclf = VotingClassifier(estimators=[('svc', clf2),('mnb', clf4), ('gnb', clf3) ], voting='hard')

for classifier in [clf3 , clf4 , eclf]:
  classifier.fit(x_train, y_train)
  y_pred = classifier.predict(x_test)
  print(classifier.score(x_test,y_test))
  print(str(classifier))
  print(classification_report(y_test, y_pred))

0.5192
GaussianNB(priors=None, var_smoothing=1e-09)
              precision    recall  f1-score   support

           0       0.51      0.98      0.67     25039
           4       0.76      0.05      0.10     24961

    accuracy                           0.52     50000
   macro avg       0.63      0.52      0.39     50000
weighted avg       0.63      0.52      0.39     50000

0.51332
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
              precision    recall  f1-score   support

           0       0.73      0.04      0.08     25039
           4       0.51      0.98      0.67     24961

    accuracy                           0.51     50000
   macro avg       0.62      0.51      0.38     50000
weighted avg       0.62      0.51      0.38     50000

0.51948
VotingClassifier(estimators=[('svc',
                              LinearSVC(C=1.0, class_weight=None, dual=True,
                                        fit_intercept=True, intercept_scaling=1,
                        

# Now time to collect some tweets based on the given query

for this task you can use multiple libraries, one of the very best ones is [twitterscraper by taspinar](https://github.com/taspinar/twitterscraper).

In [59]:
!pip install twitterscraper

     |████████████████████████████████| 92kB 3.8MB/s 
  Created wheel for twitterscraper: filename=twitterscraper-1.4.0-cp36-none-any.whl size=11347 sha256=18e07bc75af7ba9096a9e38e80818f3537f6b0700141cce5cc44255de3b5140c
  Stored in directory: /root/.cache/pip/wheels/c2/9c/8b/7393e7bdc8abe6ce0d46c2ffae2035a1a2080a97ff0ddbdde6
Successfully built twitterscraper


In [60]:
from twitterscraper import query_tweets

INFO: {'User-Agent': 'Mozilla/5.0 (compatible, MSIE 11, Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko'}


In [75]:
TestTweets = pd.DataFrame(columns=["Tweets"])
for index,tweet in enumerate(query_tweets("coronavirus", 20)[:20]):
    TestTweets.loc[index,'Tweets'] = tweet.text.encode('utf-8')[2:]
    print(tweet.text.encode('utf-8')[2:])

INFO: queries: ['coronavirus since:2006-03-21 until:2006-11-30', 'coronavirus since:2006-11-30 until:2007-08-12', 'coronavirus since:2007-08-12 until:2008-04-22', 'coronavirus since:2008-04-22 until:2009-01-02', 'coronavirus since:2009-01-02 until:2009-09-13', 'coronavirus since:2009-09-13 until:2010-05-26', 'coronavirus since:2010-05-26 until:2011-02-04', 'coronavirus since:2011-02-04 until:2011-10-17', 'coronavirus since:2011-10-17 until:2012-06-27', 'coronavirus since:2012-06-27 until:2013-03-09', 'coronavirus since:2013-03-09 until:2013-11-19', 'coronavirus since:2013-11-19 until:2014-07-31', 'coronavirus since:2014-07-31 until:2015-04-12', 'coronavirus since:2015-04-12 until:2015-12-22', 'coronavirus since:2015-12-22 until:2016-09-02', 'coronavirus since:2016-09-02 until:2017-05-14', 'coronavirus since:2017-05-14 until:2018-01-24', 'coronavirus since:2018-01-24 until:2018-10-05', 'coronavirus since:2018-10-05 until:2019-06-17', 'coronavirus since:2019-06-17 until:2020-02-27']
INFO

b' waiting for 3 10-page printouts on coronavirus for class tuesday. Hooray. Good work to both Karla and Kelly on respective bleeding stuff.'
b' waiting for 3 10-page printouts on coronavirus for class tuesday. Hooray. Good work to both Karla and Kelly on respective bleeding stuff.'
b'ishiness No not yet. Vet last yr said coz she was fighting the coronavirus it wud hav been 2 dangerous for her immune system to try.'
b'rvously awaiting further test results from the vet. It will be 24 hours. They said the coronavirus titer was very high. :/'
b'oks like she might hav a good fighting chance of recovery. Only 5% of cats with coronavirus develop FIP. Am feeling alot better abt it all'
b"st got a call from the vet. Ella has a viral infection called coronavirus. She can get better, only if it doesn't develop into the fat ..."
b'nine coronavirus S gene and uses therefor\n[5013663]'
b'oyaleagle Seen our coronavirus DB? http://lin.cr/7gq'
b'scussing coronavirus (SARS) in virology class right now'

# Now apply the model you have trained on the scraped data

In [76]:
TestTweets["Tweets"] = TestTweets["Tweets"].apply(normalize_text)
TestTweets["Tweets"] = TestTweets["Tweets"].apply(stemming_text)
TestTweets["Tweets"] = TestTweets["Tweets"].apply(lemm_text)

TestTweets

,Tweets
0,b wait for page printout on coronaviru for cla...
1,b wait for page printout on coronaviru for cla...
2,b ishi no not yet vet last yr say coz she wa f...
3,b rvousli await further test result from the v...
4,b ok like she might hav a good fight chanc of ...
5,b st get a call from the vet ella ha a viral i...
6,b nine coronaviru s gene and use therefor n
7,b oyaleagl see our coronaviru db http lin cr gq
8,b scuss coronaviru sar in virolog class right now
9,b view an aw paper on coronaviru evolut what d...


In [77]:
testtweets = vectorizer.transform(TestTweets["Tweets"])

testtweets = pd.DataFrame(
    testtweets.astype(np.int32).toarray(), columns=vectorizer.get_feature_names())
testtweets.head()

,abl,absolut,abt,ac,accept,access,accid,account,ace,ach,act,action,activ,actual,ad,adam,add,addict,address,admit,ador,adventur,advic,afford,afraid,afternoon,age,ago,agre,ah,aha,ahaha,ahead,ahh,ahhh,ahhhh,aim,ain,aint,air,...,wouldn,wouldnt,wow,wrap,write,wrong,wtf,www,xbox,xd,xo,xoxo,xx,xxx,xxxx,ya,yah,yall,yard,yay,ye,yea,yeah,year,yeh,yell,yep,yesterday,yey,yfrog,yo,york,young,youtub,youu,yr,yu,yum,yummi,yup
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [78]:
y_pred = classifier.predict(testtweets)
y_pred

array([0, 0, 0, 0, 0, 0, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

# Let's make some good looking charts 

use your favorite plotting library to plot the sentiment aggregation of the tweets you have gathered.

# Good job ! 

<center><img src="https://media.giphy.com/media/YRuFixSNWFVcXaxpmX/giphy.gif"></center>

# Now publish the notebook and show the world your work !